# Downloading Demographic and Economic Data For Regional Sales Analysis

## Population by Country

In [4]:
# Import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
# List of country codes (World Bank ISO codes)
countries = {
    "China": "CN",
    "Switzerland": "CH",
    "United States": "US",
    "Japan": "JP",
    "United Kingdom": "GB",
    "European Union": "EU",
    "Singapore": "SG",
    "Taiwan": "TW",
    "United Arab Emirates": "AE",
    "South Korea": "KR"
}

In [6]:
# Base URL for the World Bank API
wb_api_url = "http://api.worldbank.org/v2/country/{}/indicator/SP.POP.TOTL?format=json"

# Fetch data for each country
population_data = []

for country, code in countries.items():
    url = wb_api_url.format(code)
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Check if the response contains the expected data structure
        if isinstance(data, list) and len(data) > 1 and isinstance(data[1], list) and len(data[1]) > 0:
            first_entry = data[1][0]
            
            # Ensure the entry contains the required fields
            if "value" in first_entry and "date" in first_entry:
                latest_population = first_entry["value"]
                year = first_entry["date"]
                population_data.append({"Country": country, "Year": year, "Population": latest_population})
            else:
                population_data.append({"Country": country, "Year": "N/A", "Population": "Data Not Available"})
        else:
            population_data.append({"Country": country, "Year": "N/A", "Population": "No Data Found"})
    else:
        population_data.append({"Country": country, "Year": "N/A", "Population": "Error Fetching Data"})

In [7]:
# Creating DataFrame
df_population = pd.DataFrame(population_data)
df_population['Country'] = df_population['Country'].replace('European Union', 'Eurozone')

df_population


,Country,Year,Population
0,China,2023,1410710000
1,Switzerland,2023,8888093
2,United States,2023,334914895
3,Japan,2023,124516650
4,United Kingdom,2023,68350000
5,Eurozone,2023,448804042
6,Singapore,2023,5917648
7,Taiwan,N/A,No Data Found
8,United Arab Emirates,2023,10483751
9,South Korea,2023,51712619


In [8]:
# As the world bank does not provide population of Taiwan, we will enter it manually. Performing scraping for just one entry would not be efficient.
# Source: Worldometers
df_population.loc[df_population['Country'] == 'Taiwan', ['Year', 'Population']] = ['2023', 23317145]

## Currency Exchange Rate

In [9]:
# URL for exchange rates from EUR
URL = "https://www.x-rates.com/table/?amount=1&from=EUR"

# Fetch webpage content
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table", class_="tablesorter ratesTable")
currency_rates = {}

# Extract data from table rows
for row in table.find_all("tr")[1:]:  # Skip the header row
    cols = row.find_all("td")
    if len(cols) > 1:
        currency = cols[0].text.strip()
        rate = float(cols[1].text.strip())
        currency_rates[currency] = rate

# Define required currencies and their correct names from the website
currencies = {
    "Chinese Yuan Renminbi": "CNY",
    "Swiss Franc": "CHF",
    "US Dollar": "USD",
    "Japanese Yen": "JPY",
    "British Pound": "GBP",
    "Singapore Dollar": "SGD",
    "Taiwan New Dollar": "TWD",
    "Emirati Dirham": "AED",
    "South Korean Won": "KRW"
}

In [10]:
# Prepare data for DataFrame
data = []
for country, code in currencies.items():
    rate = currency_rates.get(country, "N/A")
    data.append([code, country, rate])

# Create DataFrame
df_currency = pd.DataFrame(data, columns=["Currency Code", "Country", "Exchange Rate"])

# Print the DataFrame
df_currency

,Currency Code,Country,Exchange Rate
0,CNY,Chinese Yuan Renminbi,7.595697
1,CHF,Swiss Franc,0.942424
2,USD,US Dollar,1.042608
3,JPY,Japanese Yen,158.329588
4,GBP,British Pound,0.828614
5,SGD,Singapore Dollar,1.400608
6,TWD,Taiwan New Dollar,34.178978
7,AED,Emirati Dirham,3.828979
8,KRW,South Korean Won,1503.322646


## Pushin Dataframes To Google Cloud Storage


In [ ]:
from google.cloud import bigquery
import os

# Set the service account key file
os.environ["GOOGLE_APPLICATION_CREENTIALS"] = "inlaid-chiller-442215-u9-669751b975d6.json"

# Initialize BigQuery client
client = bigquery.Client()
print("Authenticated successfully!")

Authenticated successfully!


In [ ]:
# Define full table paths
table_1_id = f"inlaid-chiller-442215-u9.Country_data.Population_data"
table_2_id = f"inlaid-chiller-442215-u9.Country_data.Currency_data"
table_3_id = f"inlaid-chiller-442215-u9.Tag_Heuer_Specs.Tag_Heuer_Specs"


# Upload first DataFrame
job_1 = client.load_table_from_dataframe(df_population, table_1_id)
job_1.result()  # Wait for completion

# Upload second DataFrame
job_2 = client.load_table_from_dataframe(df_currency, table_2_id)
job_2.result()  # Wait for completion

# Upload Third DataFrame
job_3 = client.load_table_from_dataframe(pd.read_csv('tag_heuer_features.csv', usecols=lambda column: column not in ["Power reserve (hours)"]), table_3_id)
job_3.result()  # Wait for completion

print("DataFrames uploaded successfully as new tables in BigQuery!")

Both DataFrames uploaded successfully as new tables in BigQuery!
